In [11]:
import os
import sys
import pandas as pd
import numpy as np
import time
import warnings

from tqdm import tqdm

warnings.filterwarnings("ignore")

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


sys.path.append("..")

from utils import DATA_DIR  # noqa

In [13]:
# BertのモデルとTokenizer(前処理用)をimport
from transformers import BertTokenizer, BertModel

In [ ]:
class BertClf(nn.Module):
    def __init__(self, input_size=768, num_classes=3, dropout_rate=0.3):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        output = self.dropout(x)
        output = self.linear(output)
        proba = F.softmax(output, dim=1)

        return output, proba

In [ ]:
tweet_df = pd.read_csv(os.path.join(DATA_DIR, "cleaned_airline_tweets.csv"))
tweet_df["sentiment"] = tweet_df["sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2})

train, test = train_test_split(tweet_df, test_size=0.2, random_state=42, stratify=tweet_df["sentiment"])
train, test = train.reset_index(drop=True), test.reset_index(drop=True)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else 
                      "mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [ ]:
bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = BertClf(input_size=768, num_classes=3, dropout_rate=0.3)
model = model.to(device)


RuntimeError: MPS backend out of memory (MPS allocated: 27.06 GB, other allocations: 102.84 MB, max allowed: 27.20 GB). Tried to allocate 89.42 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).